In [1]:
import pandas as pd
import os

In [2]:
raw_data_path = "../data/raw/play_by_play_2023.csv"

if not os.path.exists(raw_data_path):
    raise FileNotFoundError(f"Data file not found at: {raw_data_path}")

In [ ]:
pbp_data = pd.read_csv(raw_data_path)
pbp_data.info()

In [ ]:
pbp_data.head()

In [ ]:
missing_values = pbp_data.isnull().sum()
print("Missing values per column:")
print(missing_values)

pbp_data.describe()

In [ ]:
processed_data_path = "../data/processed/play_by_play_2023_cleaned.csv"
os.makedirs("../data/processed", exist_ok=True)
pbp_data.to_csv(processed_data_path, index=False)

print(f"Processed data saved to {processed_data_path}.")

In [ ]:
processed_data_path = "../data/processed/play_by_play_2023_cleaned.csv"
pbp_data_cleaned = pd.read_csv(processed_data_path, low_memory=False)

# Define numerical columns for median imputation
numerical_columns = ['xyac_median_yardage', 'xyac_success', 'xyac_fd']

# Fill missing numerical values with the median of each column
for col in numerical_columns:
    pbp_data_cleaned[col] = pbp_data_cleaned[col].fillna(pbp_data_cleaned[col].median())

# Define categorical columns for imputation
categorical_columns = ['xpass', 'pass_oe']

# Fill missing categorical values with the placeholder 'unknown'
for col in categorical_columns:
    pbp_data_cleaned[col] = pbp_data_cleaned[col].fillna('unknown')

# Print the count of missing values in numerical and categorical columns
print(pbp_data_cleaned[numerical_columns].isnull().sum())
print(pbp_data_cleaned[categorical_columns].isnull().sum())

In [ ]:
print(pbp_data_cleaned.describe())
print(pbp_data_cleaned['home_team'].unique())

In [ ]:
# Libraries for model training and imputation
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Define list of features and target variables for model training
features = ['qb_epa', 'yardline_100', 'xyac_success', 'game_seconds_remaining'] 
target = 'xyac_epa'

# Select rows with non-missing target values and extract features and target
X = pbp_data_cleaned.dropna(subset=[target])[features]
y = pbp_data_cleaned.dropna(subset=[target])[target]

# Divide the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model to predict the target variable
model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
model.fit(X_train, y_train)

# Predict on validation set and calculate Mean Squared Error
y_pred = model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f"Validation Mean Squared Error: {mse}")

# Identify rows with missing target values
missing_mask = pbp_data_cleaned['xyac_epa'].isna()
X_missing = pbp_data_cleaned.loc[missing_mask, features]

# Impute missing feature values and predict missing target values
if not X_missing.empty:
    # Use median imputation for missing feature values
    imputer = SimpleImputer(strategy='median')
    X_missing_imputed = pd.DataFrame(
        imputer.fit_transform(X_missing),
        columns=X_missing.columns,
        index=X_missing.index
    )

    # Predict and fill missing target values
    pbp_data_cleaned.loc[X_missing_imputed.index, 'xyac_epa'] = model.predict(X_missing_imputed)

# Print remaining missing values in the target column
print(f"Remaining missing values in 'xyac_epa': {pbp_data_cleaned['xyac_epa'].isna().sum()}")

In [ ]:
print(pbp_data_cleaned['xyac_epa'].isna().sum())

In [ ]:
missing_summary = pbp_data_cleaned.isna().sum()
print(missing_summary[missing_summary > 0])

In [ ]:
relevant_columns = ['yardline_100', 'qb_epa', 'xyac_mean_yardage']

missing_summary = pbp_data_cleaned[relevant_columns].isnull().sum()
print("Missing values per relevant column:\n", missing_summary)

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Random Forest imputation
def random_forest_impute(data, target_column, features):
    missing_mask = data[target_column].isnull()
    
    # Separate rows with and without missing target values
    data_with_values = data[~missing_mask]
    data_missing_values = data[missing_mask]

    if data_with_values.empty:
        print(f"No available data to train for {target_column}. Skipping imputation.")
        return data
    if data_missing_values.empty:
        print(f"No missing values to impute in {target_column}. Skipping imputation.")
        return data

    # Check for no missing values in the features used for training
    X = data_with_values[features].dropna()
    y = data_with_values.loc[X.index, target_column]

    if X.empty or y.empty:
        print(f"No sufficient data to train the model for {target_column}. Skipping.")
        return data

    # Organize data into training and validation sets for imputation model to use
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train Random Forest model
    model = RandomForestRegressor(random_state=42, n_estimators=100)
    model.fit(X_train, y_train)

    # Validate the model
    val_predictions = model.predict(X_val)
    val_mse = ((val_predictions - y_val) ** 2).mean()
    print(f"Validation Mean Squared Error for {target_column}: {val_mse}")

    # Predict missing values using model
    X_missing = data_missing_values[features].dropna()
    if X_missing.empty:
        print(f"No valid features available for imputation in {target_column}.")
        return data

    data.loc[missing_mask & X_missing.index, target_column] = model.predict(X_missing)

    return data

# Organize categorical and numerical columns
categorical_columns = ['posteam', 'posteam_type', 'defteam', 'side_of_field']
numerical_columns = ['yardline_100', 'qb_epa', 'xyac_mean_yardage']

# Check and filter available categorical columns
available_categorical_columns = [col for col in categorical_columns if col in pbp_data_cleaned.columns]

if not available_categorical_columns:
    print("No categorical columns available for encoding.")
else:
    # Encode categorical columns
    pbp_data_cleaned = pd.get_dummies(pbp_data_cleaned, columns=available_categorical_columns, drop_first=True)

# Update relevant columns to include encoded categorical columns
encoded_columns = [
    col for col in pbp_data_cleaned.columns if 
    any(base_col in col for base_col in available_categorical_columns)
]

# Add numerical columns to the relevant columns list
relevant_columns = encoded_columns + numerical_columns

# Inspect missing data patterns
missing_summary = pbp_data_cleaned[relevant_columns].isnull().sum()
print("Missing values per relevant column:\n", missing_summary)

# Perform Random Forest imputation for numerical columns
for column in numerical_columns:
    features = [col for col in pbp_data_cleaned.columns if col != column]
    pbp_data_cleaned = random_forest_impute(pbp_data_cleaned, column, features)

# Confirm missing values have been addressed
missing_summary_after = pbp_data_cleaned[relevant_columns].isnull().sum()
print("\nMissing values after imputation:\n", missing_summary_after)

In [13]:
pbp_data_cleaned_backup = pbp_data_cleaned.copy()

In [ ]:
from sklearn.impute import KNNImputer

# Select features and subset data for KNN imputation
knn_features = ['yardline_100', 'qb_epa', 'down', 'play_type', 'score_differential', 'game_seconds_remaining']
knn_data = pbp_data_cleaned[knn_features]

# Encode categorical variables for KNN imputation
knn_data_encoded = pd.get_dummies(knn_data, columns=['play_type'], drop_first=True)

# Apply KNN imputation
knn_imputer = KNNImputer(n_neighbors=5)
knn_data_imputed = pd.DataFrame(
    knn_imputer.fit_transform(knn_data_encoded),
    columns=knn_data_encoded.columns,
    index=knn_data.index
)

# Update the original dataset with KNN imputed values
pbp_data_cleaned['yardline_100'] = knn_data_imputed['yardline_100']
pbp_data_cleaned['qb_epa'] = knn_data_imputed['qb_epa']

# Impute xyac_mean_yardage using median
pbp_data_cleaned['xyac_mean_yardage'] = pbp_data_cleaned['xyac_mean_yardage'].fillna(pbp_data_cleaned['xyac_mean_yardage'].median())

missing_values_after_imputation = pbp_data_cleaned[['yardline_100', 'qb_epa', 'xyac_mean_yardage']].isnull().sum()
print("Missing values after imputation:")
print(missing_values_after_imputation)

In [ ]:
print(pbp_data_cleaned.columns)

In [16]:
pbp_data_cleaned_backup = pbp_data_cleaned.copy()

In [17]:
processed_data_path = "../data/processed/play_by_play_2023_cleaned.csv"
os.makedirs("../data/processed", exist_ok=True)
pbp_data.to_csv(processed_data_path, index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

processed_data_path = "../data/processed/play_by_play_2023_cleaned.csv"
pbp_data_cleaned = pd.read_csv(processed_data_path, low_memory=False)

# Define score differential column
pbp_data_cleaned['score_differential'] = pbp_data_cleaned['home_score'] - pbp_data_cleaned['away_score']

# Establish time pressure buckets (i.e. early in game, middle of game, end of game)
def time_pressure(seconds):
    if (seconds > 2400):
        return 'early_game'
    elif (2400 >= seconds >= 1200):
        return 'mid_game'
    else:
        return 'end_game'

pbp_data_cleaned['time_pressure'] = pbp_data_cleaned['game_seconds_remaining'].apply(time_pressure)

# Sort successful plays (gain 10 yards minimum on 1st down) using binary
pbp_data_cleaned['play_success'] = ((pbp_data_cleaned['yards_gained'] >= 10) & (pbp_data_cleaned['down'] == 1)).astype(int)

# Identify and sum up NaN values in 'score_differential' and 'play_success'
nan_score_diff = pbp_data_cleaned['score_differential'].isna().sum()
nan_play_success = pbp_data_cleaned['play_success'].isna().sum()

print(f"Number of NaN values in 'score_differential': {nan_score_diff}")
print(f"Number of NaN values in 'play_success': {nan_play_success}")

# One-hot encoding for 'time_pressure' and 'play_type' columns
pbp_data_encoded = pd.get_dummies(pbp_data_cleaned, columns=['time_pressure', 'play_type'], drop_first=True)

# Normalize the numerical features with MinMaxScaler
numerical_features = ['yardline_100', 'score_differential', 'game_seconds_remaining', 'qb_epa']
scaler = MinMaxScaler()
pbp_data_encoded[numerical_features] = scaler.fit_transform(pbp_data_encoded[numerical_features])

output_path = "../data/processed/play_by_play_2023_features.csv"
pbp_data_encoded.to_csv(output_path, index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Visualize the distribution 'play_success'
sns.countplot(x='play_success', data=pbp_data_encoded)
plt.title("Class Distribution of Play Success")
plt.show()

# Organize numerical columns:
numerical_columns = pbp_data_encoded.select_dtypes(include=['number']).columns
correlation_matrix = pbp_data_encoded[numerical_columns].corr()

# Visualize correlations among numerical features
plt.figure(figsize=(6, 5))
sns.heatmap(correlation_matrix, cmap="coolwarm", annot=False, fmt='.2f')
plt.title("Feature Correlation Heatmap")
plt.show()

In [20]:
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

In [ ]:
print(pbp_data_encoded.info())
print("Number of rows:", pbp_data_encoded.shape[0])
print("Number of columns:", pbp_data_encoded.shape[1])

# Check unique values in non-numeric columns
non_numeric_cols = pbp_data_encoded.select_dtypes(include=['object']).columns
for col in non_numeric_cols:
    print(f"{col}: {pbp_data_encoded[col].nunique()} unique values")

In [ ]:
# Separate features and target 
X = pbp_data_encoded.drop("play_success", axis=1)
y = pbp_data_encoded["play_success"]

# Identify non-numeric columns
non_numeric_cols = X.select_dtypes(include=['object']).columns

# Avoid over-complicating the model by reducing the number of unique categories in non-numeric columns
rare_threshold = 50  # Minimum frequency for a category to be retained
for col in non_numeric_cols:
    top_categories = X[col].value_counts().nlargest(rare_threshold).index
    X[col] = X[col].apply(lambda x: x if x in top_categories else "Other")

# Identify and drop high-cardinality columns
max_unique_categories = 500  # Threshold for acceptable cardinality
high_cardinality_cols = [col for col in non_numeric_cols if X[col].nunique() > max_unique_categories]
X = X.drop(high_cardinality_cols, axis=1)

non_numeric_cols = X.select_dtypes(include=['object']).columns

# One-hot encoding to convert categorical columns into binary columns
one_hot_encoder = OneHotEncoder(sparse_output=True, drop='first')

# Form a sparse matrix for storage efficiency
X_encoded_sparse = one_hot_encoder.fit_transform(X[non_numeric_cols])

# Convert sparse matrix to DataFrame
X_encoded = pd.DataFrame.sparse.from_spmatrix(
    X_encoded_sparse,
    columns=one_hot_encoder.get_feature_names_out(non_numeric_cols),
    index=X.index
)

# Remove original non-numeric columns and concatenate encoded columns
X = pd.concat([X.drop(non_numeric_cols, axis=1), X_encoded], axis=1)

# Drop columns full of NaN values
X = X.dropna(axis=1, how='all')

# Handle missing values by imputing with median
imputer = SimpleImputer(strategy="median")  
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Use SMOTE for balancing data with equal repesentation of classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print(f"Shape of X after encoding and SMOTE: {X_resampled.shape}")
print(f"Number of features: {X_resampled.shape[1]}")

processed_filename = '../data/processed/play_by_play_2023_smote_processed.csv'
pd.concat([X_resampled, y_resampled], axis=1).to_csv(processed_filename, index=False)

print(f"Processed data saved as {processed_filename}")

In [ ]:
import matplotlib.pyplot as plt

# Visualize class distribution after applying SMOTE
plt.figure(figsize=(8, 6))
y_resampled.value_counts().plot(kind="bar", color=["skyblue", "orange"])
plt.title("Class Distribution After SMOTE")
plt.xlabel("Class")
plt.ylabel("Count")

# Select first five features for visualization 
feature_subset = X_resampled.iloc[:, :5]

# Plot feature distributions
feature_subset.hist(figsize=(12, 8), bins=20, color='dodgerblue', edgecolor='black')
plt.suptitle("Distributions of Selected Features", fontsize=16)
plt.show()

In [3]:
import pandas as pd
csv_path = "/tmp/play_by_play_2023_filtered.csv"
df = pd.read_csv(csv_path)

In [4]:
df = df.loc[:, ~df.columns.duplicated()]

In [5]:
cleaned_path = "/tmp/play_by_play_filtered_cleaned.csv"
df.to_csv(cleaned_path, index=False)